In [7]:
import pandas as pd
import pickle
import gcsfs

from count_modeling.utils import get_model_data
from count_modeling.constants import CURRENT_MODEL_DATA_VERSION, MODEL_PATH

In [3]:
#read in models
MODEL_VERSION = '2023-05-24T08:35:37.020376-07:00'

fs = gcsfs.GCSFileSystem()
with fs.open(MODEL_PATH.format(version=MODEL_VERSION), 'rb') as fi:
    models = pickle.load(fi)

In [8]:
display(pd.DataFrame(models).T)

,in_sample_rmse,out_sample_rmse,model
dummy,287.41111,172.047387,DummyRegressor()
vanilla_poisson,172.42426,147.118381,(ColumnTransformer(transformers=[('categorical...
rfr,144.519906,131.831806,(ColumnTransformer(transformers=[('categorical...
gbr,305325.418199,215748.782432,(ColumnTransformer(transformers=[('categorical...


In [9]:
best_model = models['rfr']['model']

In [10]:
X_train, X_test, y_train, y_test, context_train, context_test = get_model_data(CURRENT_MODEL_DATA_VERSION)

In [11]:
residual_df = context_train.join(X_train).join(y_train)

In [12]:
residual_df['y_pred'] = best_model.predict(X_train)
residual_df['y_resid'] = residual_df['AADB'] - residual_df['y_pred']

In [13]:
residual_df['abs_y_resid'] = residual_df['y_resid'].abs()

In [14]:
residual_df.query('abs_y_resid > 250').explore('y_resid')